## 1. 管理工具安装
### 1.1 langchain 安装

In [1]:
!pip install langchain

# conda 
# conda install -c conda-forge langchain

### 1.2 其他库安装

In [2]:
# 直接转换为 REST API
!pip install "langserve[all]"

# 观测平台
!pip install -U langsmith

# LangGraph
!pip install -U langgraph

# DeepSeek
!pip install langchain-deepseek

# 外部资源集成
!pip install langchain_community

# dotenv管理密钥，加载环境变量等
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langserve]
  Using cached langsmith-0.6.2-py3-none-any.whl.metadata (15 kB)
Using cached langsmith-0.6.2-py3-none-any.whl (282 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.53
    Uninstalling langsmith-0.4.53:
      Successfully uninstalled langsmith-0.4.53
  Using cached langgraph-1.0.5-py3-none-any.whl.metadata (7.4 kB)
  Using cached langgraph_sdk-0.3.2-py3-none-any.whl.metadata (1.6 kB)
Using cached langgraph-1.0.5-py3-none-any.whl (157 kB)
Using cached langgraph_sdk-0.3.2-py3-none-any.whl (66 kB)
  Attempting uninstall: langgraph-sdk
    Found existing installation: langgraph-sdk 0.2.12
    Uninstalling langgraph-sdk-0.2.12:
      Successfully uninstalled langgraph-sdk-0.2.12
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.4
    Uninstalling langgraph-1.0.4:
      Successfully uninstalled

### 1.3 常见的处理链包含三个要素

1. 语言模型：核心推理引擎
2. 提示词模板：提供指令
3. 输出解释器：转换为易于使用的格式，便于下游处理

#### 1.3.1 语言模型

langchain语言模型主要分为两种：

1. LLM 通用模型：接收字符串作为输入，输出字符串作为输出。
2. ChatModel 对话模型：接收消息列表，输出“消息”，用于一问一答。

消息：由 BaseMessage 类定义，包含文本、时间戳、用户标识符等属性。主要的有

1. 消息的内容：文本，通常是字符串
2. 角色：消息的发送方

角色： LangChain 用于区分不同角色的对象

1. HumanMessage：人类（用户）输入的BaseMessage。
2. AIMessage：AI助手（大模型）输出的BaseMessage。
3. SystemMessage：系统预设的BaseMessage。
4. FunctionMessage：自定义函数输出的BaseMessage。
5. ToolMessage：调用第三方工具输出的BaseMessage。
6. ChatMessage：自定义角色。

方法： LCEL 默认实现同步调用方法，最常见的是 invoke 方法，接受一个 BaseMessage 对象作为参数，并返回一个 BaseMessage 对象作为结果。

比如：
LLMs.invoke：输入输出都是字符串
ChatModels.invoke：输入输出都是BaseMessage对象

In [6]:
import os
from dotenv import load_dotenv, find_dotenv

# 加载 .env 文件
_ = load_dotenv(find_dotenv())

# 验证 Key 是否存在 (可选)
# print(os.environ.get("DASHSCOPE_API_KEY"))

In [8]:
import os
from dotenv import load_dotenv, find_dotenv

# 自动搜索并加载 .env 文件（会逐级向上查找）
env_path = find_dotenv()
loaded = load_dotenv(env_path)

print(f"Loading .env from: {env_path}")
print(f"Loaded successfully: {loaded}")

# 检查 Key 是否真的加载进来了 (只打印前几位，确保安全)
api_key = os.getenv("DASHSCOPE_API_KEY")
if api_key:
    print(f"API Key loaded: {api_key[:6]}******")
else:
    print("❌ API Key NOT found! Please check your .env file.")

Loading .env from: /Users/anthony/Desktop/llm/ai-engineer-training/week04/.env
Loaded successfully: True
API Key loaded: sk-8a3******


In [9]:
from langchain_deepseek import ChatDeepSeek

deepseek = ChatDeepSeek(model="deepseek-chat")

print(deepseek.invoke("你好"))

content='你好！很高兴见到你！😊 我是DeepSeek，一个热心的AI助手，随时准备为你提供帮助。\n\n无论你想聊天、提问、寻求建议，还是需要协助处理文档、分析问题，我都很乐意为你服务。我会尽我所能用热情和细心来回答你的每一个问题。\n\n今天有什么我可以帮助你的吗？不管是学习、工作还是生活中的任何话题，我都很期待与你交流！✨' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 5, 'total_tokens': 92, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 5}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '3131c049-1b14-4a48-ba14-7abd51972e43', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bac4c-9435-79a1-bd81-df6d4ac0337e-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 5, 'output_tokens': 87, 'total_tokens': 92, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


#### 1.3.2 提示词模板

提示词模板是用于生成提示词的模板。例如，`{name}，你好！我是你的朋友小爱同学。`就是一个提示词模板。

提示词模板中可以使用变量，变量名由字母、数字和下划线组成，且必须以字母开头。例如，`{name}，你好！我是你的朋友小爱同学。`中的`{name}`就是一个变量。

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "Hello, I am a {model_name}. How can I help you today?"
)
prompt.format(model_name="chatbot")


'Hello, I am a chatbot. How can I help you today?'

In [6]:
# 使用 ChatPromptTemplate 创建对话模板
from langchain_core.prompts import ChatPromptTemplate

# 创建包含 system 和 human 消息的聊天模板
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant named {assistant_name}."),
    ("human", "Hello, my name is {user_name}. {question}")
])

# 格式化消息
messages = chat_prompt.format_messages(
    assistant_name="Claude",
    user_name="Alice", 
    question="What's the weather like today?"
)

for message in messages:
    print(f"{message.type}: {message.content}")


system: You are a helpful AI assistant named Claude.
human: Hello, my name is Alice. What's the weather like today?


In [4]:
from langchain_core.prompts import PromptTemplate

# 多变量模板
complex_prompt = PromptTemplate(
    input_variables=["topic", "audience", "tone"],
    template="""
    请为{audience}写一篇关于{topic}的文章。
    写作风格应该是{tone}的。
    
    文章要求：
    - 内容准确且有用
    - 结构清晰
    - 适合目标受众
    """
)

formatted_prompt = complex_prompt.format(
    topic="人工智能",
    audience="初学者",
    tone="通俗易懂"
)
print(formatted_prompt)



    请为初学者写一篇关于人工智能的文章。
    写作风格应该是通俗易懂的。

    文章要求：
    - 内容准确且有用
    - 结构清晰
    - 适合目标受众
    


In [ ]:
from langchain_core.prompts import PromptTemplate

# f-string 风格、jinja2语法 ...
f_string_prompt = PromptTemplate.from_template(
    "分析以下{data_type}数据：\n{data}\n\n请提供{analysis_type}分析。"
)

result = f_string_prompt.format(
    data_type="销售",
    data="Q1销售额: 100万, Q2销售额: 120万",
    analysis_type="趋势"
)
print(result)


分析以下销售数据：
Q1销售额: 100万, Q2销售额: 120万

请提供趋势分析。


In [8]:
from langchain_core.prompts import PromptTemplate

# 带条件逻辑的模板
conditional_prompt = PromptTemplate(
    input_variables=["user_type", "question"],
    template="""
    {%- if user_type == "expert" -%}
    作为专家，请详细回答：{question}
    {%- else -%}
    请用简单易懂的方式回答：{question}
    {%- endif -%}
    """
)


#### 1.3.3 输出解析器

输出解析器是 LangChain 中的重要组件，用于将大语言模型(LLM)的原始文本输出转换为结构化数据格式，便于程序进一步处理和使用。

主要功能
- 格式化输出: 将 LLM 的自然语言输出解析为 JSON、XML、列表等结构化格式
- 类型转换: 将文本数据转换为 Python 对象（字典、列表、自定义类等）
- 数据验证: 确保输出符合预期的格式和约束条件
- 错误处理: 处理解析失败的情况，提供重试机制

常见的输出解析器类型



In [12]:
# JSON 输出解析器
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="人物姓名")
    age: int = Field(description="年龄")
    occupation: str = Field(description="职业")

parser = PydanticOutputParser(pydantic_object=PersonInfo)

# 列表输出解析器

from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
# 将 "苹果,香蕉,橙子" 解析为 ["苹果", "香蕉", "橙子"]

# 结构化输出解析器
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="answer", description="问题的答案"),
    ResponseSchema(name="source", description="答案来源")
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [13]:
# 手动使用解析器
parser = CommaSeparatedListOutputParser()

# 模拟 LLM 的原始输出
raw_output = "Python, Java, JavaScript, C++, Go"

# 解析输出
parsed_result = parser.parse(raw_output)
print("手动解析结果:", parsed_result)
# 输出: ['Python', 'Java', 'JavaScript', 'C++', 'Go']


手动解析结果: ['Python', 'Java', 'JavaScript', 'C++', 'Go']


In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_community.llms import Tongyi

# 初始化
parser = CommaSeparatedListOutputParser()
llm = Tongyi(temperature=0)

# 直接构建提示并调用
def simple_list_generation(category):
    # 手动构建提示
    prompt = f"""请列出5个{category}的例子。
{parser.get_format_instructions()}"""
    
    # 直接调用LLM
    response = llm.invoke(prompt)
    
    # 解析结果
    return parser.parse(response)


# 使用
fruits = simple_list_generation("水果")
print("水果列表:", fruits)

languages = simple_list_generation("编程语言")
print("编程语言列表:", languages)


水果列表: ['apple', 'banana', 'orange', 'grape', 'strawberry']
编程语言列表: ['Python', 'Java', 'C++', 'JavaScript', 'Ruby']


In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.tongyi import Tongyi

# 1. 创建输出解析器
parser = CommaSeparatedListOutputParser()

# 2. 获取格式化指令
format_instructions = parser.get_format_instructions()
print("格式化指令:", format_instructions)

# 3. 创建包含格式指令的提示模板
prompt = PromptTemplate(
    template="请列出5个{category}的例子。\n{format_instructions}",
    input_variables=["category"],
    partial_variables={"format_instructions": format_instructions}
)

# 4. 初始化 LLM
llm = Tongyi(temperature=0)

# 5. 创建完整的链
chain = prompt | llm | parser

# 6. 使用示例
try:
    # 调用链并获取解析后的结果
    result = chain.invoke({"category": "水果"})
    print("解析后的结果:", result)
    print("结果类型:", type(result))
    # 输出示例: ['苹果', '香蕉', '橙子', '葡萄', '草莓']
    
    # 遍历结果
    for i, fruit in enumerate(result, 1):
        print(f"{i}. {fruit}")
        
except Exception as e:
    print(f"解析错误: {e}")

# 7. 更复杂的示例 - 多个类别
categories = ["编程语言", "运动项目", "乐器"]

for category in categories:
    print(f"\n=== {category} ===")
    try:
        result = chain.invoke({"category": category})
        for item in result:
            print(f"• {item}")
    except Exception as e:
        print(f"处理 {category} 时出错: {e}")


格式化指令: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
解析后的结果: ['苹果', '香蕉', '橙子', '葡萄', '草莓']
结果类型: <class 'list'>
1. 苹果
2. 香蕉
3. 橙子
4. 葡萄
5. 草莓

=== 编程语言 ===
• Python
• Java
• C++
• JavaScript
• Ruby

=== 运动项目 ===
• 篮球
• 足球
• 网球
• 游泳
• 田径

=== 乐器 ===
• 钢琴
• 吉他
• 小提琴
• 鼓
• 长笛


In [17]:
# 购物清单
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Tongyi

# 1. 初始化组件
parser = CommaSeparatedListOutputParser()
llm = Tongyi(temperature=0)

# 2. 创建提示模板
shopping_prompt = PromptTemplate(
    template="根据{meal_type}，生成一个包含5个食材的购物清单。\n{format_instructions}",
    input_variables=["meal_type"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 3. 创建链
shopping_chain = shopping_prompt | llm | parser

# 4. 生成不同餐型的购物清单
meals = ["早餐", "午餐", "晚餐"]
shopping_lists = {}

for meal in meals:
    shopping_lists[meal] = shopping_chain.invoke({"meal_type": meal})
    print(f"{meal}购物清单: {shopping_lists[meal]}")

# 5. 格式化输出
print("\n=== 详细购物清单 ===")
for meal, items in shopping_lists.items():
    print(f"\n🍽️ {meal}:")
    for i, item in enumerate(items, 1):
        print(f"  {i}. {item}")


早餐购物清单: ['牛奶', '鸡蛋', '面包', '黄油', '燕麦片']
午餐购物清单: ['面包', '火腿', '生菜', '番茄', '奶酪']
晚餐购物清单: ['鸡胸肉', '西兰花', '红椒', '大蒜', '橄榄油']

=== 详细购物清单 ===

🍽️ 早餐:
  1. 牛奶
  2. 鸡蛋
  3. 面包
  4. 黄油
  5. 燕麦片

🍽️ 午餐:
  1. 面包
  2. 火腿
  3. 生菜
  4. 番茄
  5. 奶酪

🍽️ 晚餐:
  1. 鸡胸肉
  2. 西兰花
  3. 红椒
  4. 大蒜
  5. 橄榄油


In [ ]:
# 多场景
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Tongyi

# 通用列表生成链
parser = CommaSeparatedListOutputParser()
list_chain = (
    PromptTemplate(
        template="请列出5个{category}的{item_type}。\n{format_instructions}",
        input_variables=["category", "item_type"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    | Tongyi(temperature=0)
    | parser
)

# 多场景使用
scenarios = [
    {"category": "早餐", "item_type": "食材"},
    {"category": "办公室", "item_type": "用品"},
    {"category": "旅行", "item_type": "必需品"},
    {"category": "健身", "item_type": "器材"},
    {"category": "学习", "item_type": "工具"}
]

for scenario in scenarios:
    result = list_chain.invoke(scenario)
    print(f"{scenario['category']}{scenario['item_type']}: {result}")


早餐食材: ['鸡蛋', '牛奶', '面包', '燕麦', '香蕉']
办公室用品: ['Pen', 'Notebook', 'Stapler', 'Printer', 'Coffee Maker']
旅行必需品: ['护照', '钱包', '手机充电器', '水杯', '防晒霜']
健身器材: ['哑铃', '杠铃', '引体向上杆', '健身球', '跳绳']
学习工具: ['Anki', 'Quizlet', 'Notion', 'Duolingo', 'Khan Academy']


#### 1.3.4 使用 LCEL 的好处

1. 简洁语法: 使用管道操作符 | 创建链，代码直观易读
2. 自动类型推断: 组件间数据类型自动转换，减少手动处理
3. 并行处理: 内置并行执行能力，提高处理效率
4. 流式处理: 原生支持流式输出，适合实时应用
5. 智能缓存: 自动缓存中间结果，避免重复计算
6. 内置调试: 更好的调试和监控能力
7. 插件化: 支持自定义组件和扩展
8. 标准接口: 统一的 Runnable 接口
9. 版本管理: 支持链的版本控制和管理